In [1]:
f_func <- "./functions.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_func, '--to=script', '--stdout'), stdout = TRUE)))

f_data_loading <- "./data_loading.ipynb"
eval(parse(text = system2('jupyter', c('nbconvert', f_data_loading, '--to=script', '--stdout'), stdout = TRUE)))

Loading required package: Rsamtools

Loading required package: GenomeInfoDb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: Biostrings



The following object is masked from ‘package:base’:

    as.matrix


Warning message:
“no DISPLAY variable so Tk is not available”
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loaded TSdist v3.7.1. See ?TSdist for help, citation("TSdist") for use in publication.


Loading required package: dtplyr

Loading required package: profvis

Loading required package: ggforce

Loading required package: pracma


Attaching package: ‘pracma’


The following object is masked from ‘package:purrr’:

    cross


The following object is masked from ‘package:Hmisc’:

    ceil


Loading required package: rstatix


Attaching package: ‘rstatix’


The following object is masked from ‘package:IRanges’:

    desc


The following object is masked from ‘package:stats’:

    filter


Loading required package: ggpubr

Loading required package: jsonlite


Attaching package: ‘jsonlite’


The following object is masked from ‘package:R.utils’:

    validate


The fo

In [2]:
host_nonMT$TLEN %>% min()

[1] 35

In [3]:
samples_metadata %>% 
    filter(origin != "controls") %>% 
    filter(sample_type != "ds-wcDNA BAL") %>% 
    select(patient) %>% unlist() %>% 
    as.character() %>% unique() %>% length()

samples_metadata %>% 
    filter(sample_type == "ssLP plasma") %>% filter(origin == "IPA samples")
samples_metadata %>% 
    filter(origin != "controls") %>% 
    filter(sample_type != "ds-wcDNA BAL") %>%  
    group_by(sample_type) %>% 
    summarise(nr_samples = n())

samples_metadata %>% 
    filter(origin != "controls") %>% 
    filter(sample_type != "ds-wcDNA BAL") %>%  
    group_by(origin) %>% 
    summarise(nr_samples = n())

samples_metadata %>% 
    filter(origin != "controls") %>% 
    filter(sample_type != "ds-wcDNA BAL") %>%  
    group_by(sample_type, origin) %>% 
    summarise(nr_samples = n())

[1] 34

sample_id,origin,sample_short,patient,sample,type,sample_type
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
A05Pasp,IPA samples,Pasp,A05,plasma,ssLP,ssLP plasma
A01Pasp,IPA samples,Pasp,A01,plasma,ssLP,ssLP plasma
A02Pasp,IPA samples,Pasp,A02,plasma,ssLP,ssLP plasma
A03Pasp,IPA samples,Pasp,A03,plasma,ssLP,ssLP plasma
A05PaspVAL,IPA samples,PaspVAL,A05,plasma,ssLP,ssLP plasma
A04Pasp,IPA samples,Pasp,A04,plasma,ssLP,ssLP plasma


sample_type,nr_samples
<chr>,<int>
dsLP BAL,10
dsLP plasma,8
ssLP BAL,23
ssLP plasma,25


origin,nr_samples
<chr>,<int>
External controls,20
IPA samples,31
Internal controls,12
Other molds,3


`summarise()` has grouped output by 'sample_type'. You can override using the `.groups` argument.


sample_type,origin,nr_samples
<chr>,<chr>,<int>
dsLP BAL,IPA samples,10
dsLP plasma,IPA samples,4
dsLP plasma,Internal controls,4
ssLP BAL,External controls,11
ssLP BAL,IPA samples,11
ssLP BAL,Other molds,1
ssLP plasma,External controls,9
ssLP plasma,IPA samples,6
ssLP plasma,Internal controls,8


In [4]:
host_nonMT$sample_id %>% unique() %>% length()

[1] 66

In [5]:
#SOME NUMBERS FOR THE MANUSCRIPT
unique_sample_id_count <- host_nonMT %>% 
    mutate(sample_short = substr(sample_id, 1,3)) %>% 
    ungroup() %>% 
    summarise(unique_count = n_distinct(sample_short), .groups = "keep")
unique_sample_id_count

numbers_per_sample_type <- host_nonMT %>%
    group_by(sample) %>%
    summarise(unique_count = n_distinct(patient), .groups = "keep")
numbers_per_sample_type

host_nonMT %>% filter(sample == "plasma") %>%
    group_by(origin) %>% 
    summarise(unique_count = n_distinct(patient), .groups = "keep")

host_nonMT %>% filter(sample == "BAL") %>%
    group_by(origin) %>% 
    summarise(unique_count = n_distinct(patient), .groups = "keep")

average_fragment_length <- host_nonMT %>% 
    group_by(TLEN, sample_type, type) %>% summarise(mnsCount = mean(nsCount, na.rm = TRUE), .groups = "keep") %>% 
    group_by(sample_type, type) %>% 
    summarise(avg_TLEN = round(sum(TLEN * mnsCount, na.rm = TRUE) / sum(mnsCount, na.rm = TRUE), digits = 0), 
              .groups = "keep")
average_fragment_length

unique_count
<int>
34


sample,unique_count
<chr>,<int>
BAL,23
plasma,16


origin,unique_count
<chr>,<int>
External controls,9
IPA samples,5
Internal controls,6
Other molds,2


origin,unique_count
<chr>,<int>
External controls,11
IPA samples,11
Other molds,1


sample_type,type,avg_TLEN
<chr>,<chr>,<dbl>
dsLP BAL,dsLP,312
dsLP plasma,dsLP,241
ssLP BAL,ssLP,182
ssLP plasma,ssLP,167


In [6]:
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP plasma", tlen_min = 170)[[1]] %>% 
    filter(TLEN >160) %>% filter(type == "peak")
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP plasma", tlen_min = 170)[[1]] %>% 
    filter(TLEN >160) %>% filter(type == "peak")
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP BAL", tlen_min = 170)[[1]] %>% 
    filter(TLEN >160) %>% filter(type == "peak")
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP BAL", tlen_min = 170)[[1]] %>% 
    filter(TLEN >160) %>% filter(type == "peak")

#get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP plasma")[[1]]
#get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP BAL")[[1]]
#get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP BAL")[[1]]

get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP plasma")[[2]]
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP plasma")[[2]]
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "ssLP BAL")[[2]]
get_index_for_valley(host_nonMT_summarized, sample_type_tmp = "dsLP BAL")[[2]]

TLEN,sample_type,type,mnsCount
<int>,<chr>,<chr>,<dbl>
166,ssLP plasma,peak,2.333132


TLEN,sample_type,type,mnsCount
<int>,<chr>,<chr>,<dbl>
166,dsLP plasma,peak,1.756357


TLEN,sample_type,type,mnsCount
<int>,<chr>,<chr>,<dbl>
164,ssLP BAL,peak,0.8138056


TLEN,sample_type,type,mnsCount
<int>,<chr>,<chr>,<dbl>
168,dsLP BAL,peak,0.2600398


sample_type,mean_index,index_non_na,index_na,total_index
<chr>,<dbl>,<int>,<int>,<int>
ssLP plasma,1.53,10,1,11


sample_type,mean_index,index_non_na,index_na,total_index
<chr>,<dbl>,<int>,<int>,<int>
dsLP plasma,1.52,4,3,7


sample_type,mean_index,index_non_na,index_na,total_index
<chr>,<dbl>,<int>,<int>,<int>
ssLP BAL,1.7,10,0,10


sample_type,mean_index,index_non_na,index_na,total_index
<chr>,<dbl>,<int>,<int>,<int>
dsLP BAL,1.54,9,1,10


In [7]:
rbind(plasma_sizeSelection_meta, BAL_sizeSelection_meta) %>%
    filter(origin != "control", 
           taxName == "Aspergillus fumigatus") %>%
    select(sample_id, new_Filter, QC_number_reads, sample_type, origin, type) %>% 
    filter(new_Filter %in% c("noFilter", "35-1000")) %>%     
    pivot_wider(names_from = new_Filter, values_from = QC_number_reads, names_prefix = "QC#_") %>% 
    mutate(perc_merged = `QC#_35-1000`/`QC#_noFilter`) %>% 
    group_by(sample_type) %>% 
    summarize(mean_perc_merged = mean(perc_merged))

sample_type,mean_perc_merged
<chr>,<dbl>
dsLP BAL,0.3126241
dsLP plasma,0.5873901
ssLP BAL,0.7933704
ssLP control,0.2592045
ssLP plasma,0.8795137
